16 Day Forecast 

Location: Dublin Castle 
- 53.342443
- 6.266724

API Key

https://api.openweathermap.org/data/2.5/onecall?lat=53.342443&lon=6.266724&exclude=minutely&appid=f800ca2d2726566b082da6aad95fe093

In [6]:
import datetime
import traceback
import os
import requests
import time
import json
import pyodbc
import textwrap
import pytz

In [7]:
# driver
driver = '{ODBC Driver 17 for SQL Server}'

In [8]:
# server and db name
server_name = 'dublinbus-team12-server'
database_name = 'dublinbus-team12-db'

In [9]:
# server
server = '{}.database.windows.net,1433'.format(server_name)

In [10]:
# username and passord 
username = 'innovationgeeks'
password = 'Laurawillsaveus445!'

In [11]:
# connection string 
connection_string = textwrap.dedent('''
    Driver={driver};
    Server={server};
    Database={database};
    Uid={username};
    Pwd={password};
    Encrypt=yes;
    TrustServerCertificate=no;
    Connection Timeout=30;
'''.format(
    driver = driver, 
    server = server, 
    database = database_name,
    username = username,
    password = password
    )                               
)

In [12]:
# pyodbc conncetion object
conn: pyodbc.Connection = pyodbc.connect(connection_string)

In [13]:
# curser object for connection 
cursor: pyodbc.Cursor = conn.cursor()

In [23]:
# current waether 
SixteenDay = '''
   CREATE TABLE weather_SixteenDayForcast(
    lon FLOAT,
    lat FLOAT,
    time_queried DATETIME,
    last_update DATETIME,
    temp_day FLOAT,
    temp_night FLOAT,
    temp_eve FLOAT,
    temp_morn FLOAT,
    feels_like_day FLOAT,
    feels_like_night FLOAT,
    feels_like_eve FLOAT,
    feels_like_morn FLOAT,
    pressure FLOAT,
    humidity FLOAT,
    weather_main VARCHAR(256),
    weather_description VARCHAR(256),
    speed FLOAT,
    deg FLOAT,
    clouds FLOAT,
    pop FlOAT,
   )'''

In [24]:
def create_table():
    '''function to create table
        - Only needs to run once'''
    try:
        cursor.execute(SixteenDay)
        cursor.commit()
    except Exception as e: 
        print(e)

In [25]:
# Call create table - Only once!
#create_table()

In [27]:
def get_data(data):
    """
    this function takes in the data and station number as input and returns a tuple with that data
    """
    now = datetime.datetime.utcnow()
    
    return (
            data.get("coord").get("lon"),
            data.get("coord").get("lat"),
            now, 
            datetime.datetime.fromtimestamp(data.get("dt")),  
            data.get("temp").get("day"),   
            data.get("temp").get("night"),   
            data.get("temp").get("eve"),   
            data.get("temp").get("morn"),   
            data.get("feels_like").get("day"),    
            data.get("feels_like").get("night"),    
            data.get("feels_like").get("eve"),     
            data.get("feels_like").get("morn"),    
            data.get("pressure"),    
            data.get("humidity"),   
            data.get("weather")[0].get("main"),    
            data.get("weather")[0].get("description"),   
            data.get("speed"),   
            data.get("deg"),   
            data.get("clouds"),   
            data.get("pop"))

In [ ]:
def SixteenDay():
    ''' Funntion to get data from database and import it '''
    r = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat=53.342443&lon=6.266724&exclude=minutely&appid=f800ca2d2726566b082da6aad95fe093")
    data = json.loads(r.text)
    days = data.get("list")
    print(data)
    for day in days:
        vals = get_data(day) 
        print(vals)
        cursor.execute("INSERT INTO `weather_SixteenDayForcast` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
        cursor.commit()
    return

In [ ]:
def main():
    while True:
        try:
            SixteenDay()
            time.sleep(60 * 60 * 24)
        except:
            print(traceback.format_exc())
            print("Error found an error when querying")
            time.sleep(60 * 60 * 24)

if __name__ == "__main__":
    main()